In [ ]:
import numpy as np
import dolfinx
from mpi4py import MPI
from petsc4py import PETSc
import ufl
from dolfinx.mesh import create_rectangle, create_mesh
from dolfinx.fem import Function, functionspace, Constant, form
from dolfinx.fem.petsc import LinearProblem
import matplotlib.pyplot as plt
import gmsh
from dolfinx.io import gmshio
from mpi4py import MPI


In [27]:


gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)


mesh_size = 0.019

gmsh.option.setNumber("Mesh.CharacteristicLengthMax", mesh_size)
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", mesh_size/2)

def gmsh_sphere(model, name: str):
    """
    Create a Gmsh model of a unit sphere and generate a surface mesh.
    This function creates a sphere (as a volume), then extracts its boundary surfaces,
    defines a physical group for them, and generates a 2D mesh.
    """
    model.add(name)
    model.setCurrent(name)

    sphere = model.occ.addSphere(0, 0, 0, 1)
    model.occ.synchronize()

    boundaries = model.getBoundary([(3, sphere)], oriented=False)
    boundary_tags = [b[1] for b in boundaries if b[0] == 2]

    model.add_physical_group(2, boundary_tags)

    model.mesh.generate(2)
    return model


model = gmsh.model()
gmsh_sphere(model, "SphereSurface")

mesh, cell_tags, facet_tags = gmshio.model_to_mesh(model, MPI.COMM_WORLD, 0)




gmsh.finalize()

Display Mesh to Check 

In [ ]:
import pyvista
from dolfinx.plot import vtk_mesh


topology, cell_types, geometry= vtk_mesh(mesh, mesh.topology.dim)
display_grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)

plotter = pyvista.Plotter()
plotter.add_mesh(display_grid, show_edges=True)
plotter.show()

In [30]:

import os
import meshio
out_dir = "data"
os.makedirs(out_dir, exist_ok=True)
out_filename = os.path.join(out_dir, "sphere_surface_test_new.vtu")
display_grid.save(out_filename)

In [ ]:


gamma_s = 10.0
r_s = 0.001
dt = 1e-3
T = 0.3
num_time_steps = int(T / dt)




V = functionspace(mesh, ("CG", 1))

t = Constant(mesh, 0.0)


phi_n = Function(V)

phi_np1 = Function(V)

phi_nm1 = Function(V)

u_ref_dofs = np.zeros((num_time_steps, V.dofmap.index_map.size_local), dtype=np.float64)
Qs_dofs = np.zeros((num_time_steps, V.dofmap.index_map.size_local), dtype=np.float64)
t_star = np.linspace(0, T, num_time_steps)

Q0 = 0.05
alpha = 1.0
Nf = 17
Ns = 20
v_const = 1.0

freqs = np.arange(1, 85, 5)
freqs = freqs[:Nf]

rng = np.random.default_rng(42)
r0_sources = rng.uniform(-1.0, 1.0, size=(Ns, 3))
v_sources = rng.uniform(-1.0, 1.0, size=(Ns, 3))


def phase_func(r, r0, vs, time):
    r_s_of_t = r0 + vs * time
    diff = r.T - r_s_of_t
    dist = np.linalg.norm(diff, axis=1)
    return dist / v_const

def pink_noise_signal(x, time):
    out = np.zeros(x.shape[1], dtype=np.float64)
    for f in freqs:
        f_term = f**(-alpha)
        for s in range(Ns):
            r0_s = r0_sources[s]
            v_s = v_sources[s]
            Pi_vals = phase_func(x, r0_s, v_s, time)
            out += f_term * np.sin(2 * np.pi * f * (time + Pi_vals))
    norm_factor = Q0 / (len(freqs) * Ns)
    return norm_factor * out

def Q_callable(x):
    return pink_noise_signal(x, t.value)

Q_function = Function(V)


u_ufl = ufl.TrialFunction(V)

v_ufl = ufl.TestFunction(V)

A_form = (
    (1 / (gamma_s**2 * dt**2) + 1/(gamma_s * dt) + 1) * ufl.inner(u_ufl, v_ufl)
    + r_s**2 * ufl.dot(ufl.grad(u_ufl), ufl.grad(v_ufl))
)*ufl.dx


for n in range(num_time_steps):

    t.value = (n + 1) * dt
    Q_function.interpolate(lambda x: Q_callable(x))




    L_form = (
            (Q_function
             + (2 / (gamma_s**2 * dt**2)) * phi_n
            + (-1 / (gamma_s**2 * dt**2) + 1 /(gamma_s * dt))  * phi_nm1
             ) * v_ufl
        ) * ufl.dx
    problem = LinearProblem(A_form, L_form)
    phi_np1.x.array[:] = problem.solve().x.array

    u_ref_dofs[n, :] = phi_np1.x.array
    Qs_dofs[n, :] = Q_function.x.array

    phi_nm1.x.array[:] = phi_n.x.array
    phi_n.x.array[:] = phi_np1.x.array

    if n % 100 == 0:
        print(f"Step {n}, t={t.value:.3f}, max(phi)={phi_np1.x.array.max()}, min(phi)={phi_np1.x.array.min()}")

coords_3d = mesh.geometry.x
coords_2d = coords_3d[:, :2]



In [32]:
output_path = "delta_pinss_test_new_big.npy"

data_dict = {
    "mesh_coordinates": coords_3d,
    "t_star": t_star,
    "phi_e": u_ref_dofs,
    "Qs": Qs_dofs
}

np.save(output_path, data_dict, allow_pickle=True)

In [ ]:


coords = mesh.geometry.x
phi_vals = phi_np1.x.array


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2],
                c=phi_vals, cmap='viridis', s=20)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('Solution phi on the Unit Sphere')
fig.colorbar(sc, ax=ax, label='phi')
plt.show()
